In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [22]:
data = pd.read_csv('/content/train.csv')

In [23]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
Y = data['label']
X = data.drop('label', axis= 1)
X = np.array(X)
Y = np.array(Y)

In [25]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, shuffle= True,test_size= 0.2)

In [26]:
type(train_x)


numpy.ndarray

In [27]:
train_x = (train_x / 255.) #RGB

In [28]:
type(train_x)

numpy.ndarray

In [29]:
m, n = train_x.shape


In [30]:
train_y

array([2, 3, 6, ..., 9, 8, 7])

In [31]:
def init_params(input_n, middle_n, output_n, classes_n= 10):
    W1 = np.random.rand(middle_n, input_n) * 0.01
    b1 = np.zeros((middle_n, 1))
    
    W2 = np.random.rand(output_n, middle_n) * 0.01
    b2 = np.zeros((output_n, 1))
    
    W3 = np.random.rand(classes_n, output_n) * 0.01
    b3 = np.zeros((classes_n, 1))
    return W1, b1, W2, b2, W3, b3


In [32]:
def ReLU(Z, der= False):
    if der == False:
        return np.maximum(Z, 0)
    else:
        return Z > 0

In [33]:
def sigmoid(Z, der= False): 
    if der == False:
        return (1/(1 + np.exp(-Z)))
    else:
        return (1 - sigmoid(Z))

In [34]:
def softmax(Z): 
        return (np.exp(Z) / sum(np.exp(Z)))

In [35]:
def forward_prop(W1, b1, W2, b2, W3, b3, X):
    # node 1
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    
    # node 2
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    # node 3
    Z3 = np.dot(W3, A2) + b3
    A3 = softmax(Z3)
    
    return Z1, A1, Z2, A2, Z3, A3

In [36]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [37]:
def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    one_hot_Y = one_hot(Y)
    
    dZ3 = A3 - one_hot_Y
    dW3 = 1 / m * np.dot(dZ3, A2.T)
    db3 = 1 / m * np.sum(dZ3, axis= 1, keepdims= True)
    
    dZ2 = np.dot(W3.T, dZ3) * sigmoid(Z2, True)
    dW2 = 1 / m * np.dot(dZ2, A1.T)
    db2 = 1 / m * np.sum(dZ2, axis= 1, keepdims= True)
    
    dZ1 = np.dot(W2.T, dZ2) * sigmoid(Z1, True)
    dW1 = 1 / m * np.dot(dZ1, X.T)
    db1 = 1 / m * np.sum(dZ1, axis= 1, keepdims= True)

    return dW1, db1, dW2, db2, dW3, db3

In [38]:
def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2
    
    W3 = W3 - alpha * dW3  
    b3 = b3 - alpha * db3
    
    return W1, b1, W2, b2, W3, b3

In [39]:
def get_predictions(A3):
    return np.argmax(A3, 0)

In [40]:
def get_accuracy(predictions, Y):
    print(predictions, Y)
    return (np.sum(predictions == Y) / Y.size) 

In [61]:
def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2, W3, b3 = init_params(train_x.T.shape[0], 100, 10)
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2, W3, b3

In [62]:
W1, b1, W2, b2, W3, b3 = gradient_descent(train_x.T, train_y, 0.10, 300)

Iteration:  0
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  10
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  20
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  30
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  40
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  50
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  60
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  70
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  80
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  90
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  100
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  110
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  120
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Iteration:  130
[1 1 1 ... 1 1 1] [2 3 6 ... 9 8 7]
0.11094943615860313
Ite

In [63]:
def make_predictions(X, W1, b1, W2, b2, W3, b3):
    _, _, _, _, _, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
    predictions = get_predictions(A3)
    return predictions


In [64]:
def test_prediction(index, W1, b1, W2, b2, W3, b3):
    current_image = train_x[index,:, None].T
    print(current_image.shape)
    prediction = make_predictions(train_x[index,:, None], W1, b1, W2, b2, W3, b3)
    label = train_y[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()


In [65]:
def F1_score(y,y_hat):
    tp,tn,fp,fn = 0,0,0,0
    for i in range(len(y)):
        if y[i] == 1 and y_hat[i] == 1:
            tp += 1
        elif y[i] == 1 and y_hat[i] == 0:
            fn += 1
        elif y[i] == 0 and y_hat[i] == 1:
            fp += 1
        elif y[i] == 0 and y_hat[i] == 0:
            tn += 1
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1_score = 2*precision*recall/(precision+recall)
    return f1_score,precision,recall

In [69]:
y_hat = make_predictions(test_x.T, W1, b1, W2, b2, W3, b3)
print('Accuracy is ', get_accuracy(y_hat, test_y))




[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [67]:
f1_score, percision, recall = F1_score(test_y, y_hat)

In [68]:
print(f1_score, percision, recall)

0.6968325791855203 0.5347222222222222 1.0
